## Check your connection to Generative AI Hub


☝️ In the init_env.py file the values from `generative-ai-codejam/.aicore-config.json` are assigned to environmental variables. That way the Generative AI Hub [Python SDK](https://pypi.org/project/generative-ai-hub-sdk/) will connect to Generative AI Hub.

👉 For the Python SDK to know which resource group to use, you also need to set the `resource group` in the [`variables.py`](variables.py) file to your own `resource group` (e.g. **team-01**) that you created in the SAP AI Launchpad in exercise [00-connect-AICore-and-AILaunchpad](00-connect-AICore-and-AILaunchpad.md).

In [9]:
import init_env
import variables
import importlib
variables = importlib.reload(variables)

# TODO: You need to specify which model you want to use. In this case we are directing our prompt
# to the openAI API directly so you need to pick one of the GPT models. Make sure the model is actually deployed
# in genAI Hub. You might also want to chose a model that can also process images here already. 
# E.g. 'gpt-4.1-mini'
MODEL_NAME = 'gpt-4.1-mini'
# Do not modify the `assert` line below
assert MODEL_NAME!='', """You should change the variable `MODEL_NAME` with the name of your deployed model (like 'gpt-4o-mini') first!"""

init_env.set_environment_variables()
# Do not modify the `assert` line below 
assert variables.RESOURCE_GROUP!='', """You should change the value assigned to the `RESOURCE_GROUP` in the `variables.py` file to your own resource group first!"""
print(f"Resource group is set to: {variables.RESOURCE_GROUP}")

Resource group is set to: default


## Prompt an LLM in the Generative AI Hub...

...using OpenAI native client integration: https://help.sap.com/doc/generative-ai-hub-sdk/CLOUD/en-US/_reference/gen_ai_hub.html#openai

To understand the API and structures, check OpenAI documentation: https://platform.openai.com/docs/guides/text?api-mode=chat

In [10]:
from gen_ai_hub.proxy.native.openai import chat
from IPython.display import Markdown

messages = [
    {
        "role": "user", 
        "content": "What is the underlying model architecture of an LLM? Explain it as short as possible."
    }
]

kwargs = dict(model_name=MODEL_NAME, messages=messages)
response = chat.completions.create(**kwargs)
display(Markdown(response.choices[0].message.content))

An LLM is typically based on the Transformer architecture, which uses layers of self-attention mechanisms and feed-forward neural networks to process and generate language by modeling token relationships in sequences.

## Understanding roles
Most LLMs have the roles `system`, `assistant` (GPT) or `model` (Gemini) and `user` that can be used to steer the models response. In the previous step you only used the role `user` to ask your question. 

👉 Try out different `system` messages to change the response. You can also tell the model to not engage in smalltalk or only answer questions on a certain topic. Then try different user prompts as well!

Please note, that in OpenAI API with `o1` models and newer, `developer` messages replace the previous `system` messages.

In [11]:
messages = [
    {   "role": "system", 
        # TODO try changing the system prompt
        "content": "Speak like a Pirate."
    }, 
    {
        "role": "user", 
        "content": "What is the underlying model architecture of an LLM? Explain it as short as possible."
    }
]

kwargs = dict(model_name=MODEL_NAME, messages=messages)
response = chat.completions.create(**kwargs)
display(Markdown(response.choices[0].message.content))

Arrr! The underlying model architecture o' an LLM be the Transformer, a mighty neural network that uses self-attention to weigh the importance o' each word in a sentence, makin' it savvy at understandin' and generatin' language, like a clever pirate readin' the winds! Yarrr!

👉 Also try to have it speak like a pirate.

👉 Now let's be more serious! Tell it to behave like an SAP consultant talking to AI Developers.

👉 Ask it to behave like an SAP Consultant talking to ABAP Developers and to make ABAP comparisons.

## Hallucinations
👉 Run the following question.

In [12]:
messages = [
    {   "role": "system", 
        "content": "You are an SAP Consultant."
    }, 
    {
        "role": "user", 
        "content": "How does the data masking of the orchestration service work?"
    }
]

kwargs = dict(model_name=MODEL_NAME, messages=messages)
response = chat.completions.create(**kwargs)
display(Markdown(response.choices[0].message.content))

Data masking in the context of an orchestration service—such as one used within SAP Integration Suite or SAP Cloud Platform Integration—typically refers to the process of concealing sensitive data elements in message payloads during processing or monitoring, to ensure data privacy and compliance.

Here is how data masking generally works in an orchestration service:

1. **Identification of Sensitive Fields:**  
   The orchestration service is configured with rules or policies that specify which data fields should be masked. These are usually sensitive elements like personally identifiable information (PII), financial data, or credentials.

2. **Masking Technique:**  
   When a message passes through the orchestration, the service applies masking techniques such as:
   - Replacing the actual values with a fixed mask like asterisks (e.g., *****)
   - Partial masking, where part of the value remains visible (e.g., last 4 digits of a credit card)
   - Substitution with generic or randomized characters

3. **Masking Scope:**  
   Masking can be applied at different stages:
   - **Logging and Monitoring:** To prevent sensitive data from being stored in logs or visible in monitoring tools.
   - **Message Content:** To ensure that downstream services or recipients only receive masked data if appropriate.
   
4. **Configuration:**  
   In SAP Integration Suite (CPI), data masking can be set up via:
   - Data store configurations
   - Content modifier steps that replace values in the message payload
   - Custom scripts (Groovy/JavaScript) that selectively mask data dynamically
   - Data protection policies that automatically redact or obfuscate sensitive information during message processing or audit logging

5. **Security and Compliance:**  
   Data masking helps meet regulatory requirements like GDPR or HIPAA by preventing exposure of confidential data during integration or message handling.

**Summary:**  
Data masking in orchestration services hides sensitive data fields in the message content or logs by applying predefined rules or scripts, ensuring that sensitive information is not exposed during message processing, monitoring, or logging. This maintains security and compliance without impacting the overall message flow.

If you’re working with a specific SAP orchestration service (e.g., SAP Integration Suite), I can provide detailed steps or examples on how to implement data masking in that environment. Let me know!

☝️ Compare the response to [SAP Help - Generative AI Hub SDK](https://help.sap.com/doc/generative-ai-hub-sdk/CLOUD/en-US/_reference/orchestration-service.html). 

👉 What did the model respond? Was it the truth or a hallucination?

👉 Which questions work well, which questions still do not work so well?

# Use Multimodal Models

Multimodal models can use different inputs such as text, audio and [images](https://platform.openai.com/docs/guides/images-vision?api-mode=chat&format=base64-encoded#giving-a-model-images-as-input). In Generative AI Hub on SAP AI Core you can access multiple multimodal models (e.g. `gpt-4o-mini`).

👉 If you have not deployed one of the gpt-4o-mini models in previous exercises, then go back to the model library and deploy a model that can also process images.

👉 Now run the code snippet below to get a description for [the image of the AI Foundation Architecture](documents/ai-foundation-architecture.png). These descriptions can then for example be used as alternative text for screen readers or other assistive tech.

👉 You can upload your own image and play around with it!

In [13]:
import base64

# get the image from the documents folder
with open("documents/ai-foundation-architecture.png", "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')

messages = [{
            "role": "user", 
             "content": [
                {"type": "text", 
                 "text": "Describe the images as an alternative text."},
                {"type": "image_url", 
                 "image_url": {
                    "url": f"data:image/png;base64,{image_data}"
                    }
                }
            ]
        }]

kwargs = dict(model_name=MODEL_NAME, messages=messages)
response = chat.completions.create(**kwargs)

display(Markdown(response.choices[0].message.content))

Alternative text:

A technical architecture diagram titled "Single-tenant application (Retrieval Augmented Generation & Generative AI on SAP BTP)" illustrates the interaction of components on the SAP Business Technology Platform (BTP) for deploying AI solutions. 

On the left, a "User" icon represents users accessing "Application Clients" via mobile or desktop devices. These interact with an "HTML5 App Repository" and "App Router" inside a box labeled "Subaccount Multi-Cloud" on SAP BTP. The App Router is connected to "SAP Authorization and Trust Management service" denoted by green arrows labeled "Trust".

Within the Subaccount box, components are divided into:
- User Interface: contains "SAPUI5" and "UI5 Web Components".
- SAP Cloud Application Programming Model (CAP): includes "Application Service" with items "Use Case Logic," "Data Management," "LLM Plugins & SDKs," and "SAP Cloud SDK for AI."
- SAP HANA Cloud: features a "Vector Engine" with a grayed-out "Knowledge Graph Engine."
- Generative AI Hub: shows multiple layers:
  - "SAP AI Launchpad"
  - "SAP AI Core" with "Trust & Control" and "Prompt Registry"
  - "Orchestration" containing "Grounding," "Templating," "Data Masking," and "I/O Filtering"
  - "Foundation Model Access": distinguishing "Partner built" and grayed-out "SAP built"
  - "Foundation Models SAP hosted"

Arrows indicate data flows and interactions: 
- From Application Clients via Destination to CAP
- From CAP to SAP HANA Cloud
- From CAP and SAP HANA Cloud to Generative AI Hub
- Two-way HTTPS connections from Generative AI Hub to three external network resources on the right:
  - SAP On-Premise Solutions via Cloud Connector
  - 3rd Party Applications
  - SAP Cloud Solutions

Legend describes:
- Dark blue circles for Access
- Green arrows for Mutual Trust
- Blue icons for SAP BTP Service
- "xyz" as planned components.

At the bottom, a note states that the diagram is Level L2 and explains it shows how to combine large language models using the Generative AI Hub in SAP AI Core.

# Extracting text from images
Nora loves bananabread and thinks recipes are a good example of how LLMs can also extract complex text from images, like from [a picture of a recipe of a bananabread](documents/bananabread.png). Try your own recipe if you like :)

This exercise also shows how you can use the output of an LLM in other systems, as you can tell the LLM how to output information, for example in JSON format.

In [14]:
# get the image from the documents folder
with open("documents/bananabread.png", "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')

messages = [{"role": "user", 
             "content": [
                {"type": "text", 
                "text": "Extract the ingredients and instructions in two different json files."},
                {"type": "image_url", 
                 "image_url": {
                    "url": f"data:image/png;base64,{image_data}"
                    }
                }
            ]
        }]

kwargs = dict(model_name=MODEL_NAME, messages=messages)
response = chat.completions.create(**kwargs)

display(Markdown(response.choices[0].message.content))

Here are the extracted ingredients and instructions in two different JSON files:

**ingredients.json**
```json
{
  "dry_ingredients": {
    "All-purpose Flour": "260 g",
    "Sugar": "200 g",
    "Baking Soda": "6 g",
    "Salt": "3 g"
  },
  "wet_ingredients": {
    "Banana": "225 g",
    "Large Eggs": "2",
    "Vegetable Oil": "100 g",
    "Whole Milk": "55 g",
    "Vanilla Extract": "5 g"
  },
  "toppings": {
    "Chocolate Chips": "100 g",
    "Walnuts (Optional)": "100 g"
  }
}
```

**instructions.json**
```json
{
  "directions": [
    "Preheat oven to 180 °C",
    "Mash banana in a bowl",
    "Combine banana and other wet ingredients together in the same bowl",
    "Mix all dry ingredients together in a separate bowl",
    "Use a whisk to combine dry mixture into wet mixture until smooth",
    "Pour mixture into a greased/buttered loaf pan",
    "Place the pan into preheated oven on the middle rack and bake for about 60 minutes or until a toothpick comes out clean",
    "Enjoy!"
  ]
}
```

[Next exercise](04-create-embeddings.ipynb)
